In [1]:
import re
import string
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from nltk import bigrams
from nltk.tokenize import sent_tokenize
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
import spacy
nlp = spacy.load('en_core_web_md')

Цього тижня ви працюватимете над задачею виправлення помилок.

Run-on речення - це речення, склеєне з двох чи більше речень без належної пунктуації. Таку помилку часто допускають механічно, коли швидко друкують текст, проте така помилка виникає і від незнання мови. Особливо часто ця помилка зустрічається в інтернет-спілкуванні.

Наприклад:

>Thanks for talking to me let's meet again tomorrow Bye.

У цьому реченні насправді три склеєні речення. Правильний варіант:

>Thanks for talking to me. Let's meet again tomorrow. Bye.

Дані:

    Виберіть будь-який відкритий корпус та згенеруйте тренувальні дані для моделі. Тренувальними даними буде набір слеєних речень. Візьміть до уваги, що склеєних речень може бути кілька (зазвичай 2, але буває і 3-4), а перше слово наступного речення може писатися з великої чи малої літери.
    Зберіть чи знайдіть у відкритому доступі базу енграмів. Візьміть до уваги, що енграми на межі речень доведеться збирати власноруч.

Тестування:

    Напишіть бейзлайн та метрику для тестування якості.
    
Класифікатор:

    Виділіть ознаки, які впливають на те, чи є слово на межі речень. Подумайте про контекст, написання слів, граматичні ознаки, енграми, межі речень тощо.
    Побудуйте класифікатор на основі логістичної регресії з використанням виділених ознак, який анотує послідовно слова у реченні на предмет закінчення речення.
    Спробуйте покращити якість роботи класифікатора, змінюючи набір чи комбінацію ознак.

In [3]:
import json
test_set = json.load(open('run-on-test.json'))

In [4]:
overall_count = 0
upcount = 0
for d in test_set:
    for i, w in enumerate(d):
        if w[1] == True:
            overall_count += 1
            if d[i+1][0].istitle():
                upcount += 1
print(overall_count, upcount)

155 75


In [5]:
from collections import Counter
counts = []
for d in test_set:
    true_count = 0
    for w in d:
        if w[1] == True:
            true_count += 1
    counts.append(true_count)
    
Counter(counts).most_common()

[(1, 145), (0, 50), (2, 5)]

In [6]:
# we have to reconstruct tokenized sentences
def reconstruct(token_list):
    """
    Make sentence from a list of tokens
    """
    text = ''
    tokens = [t[0] for t in token_list]
    for i, token in enumerate(tokens):
        if ((i != 0) 
            and (token not in '!%),.:;?}')
            and not (token.startswith("'") and len(token) > 1)):
            text += ' ' + token
        else:
            text += token
    return text

In [6]:
def make_spaceable(token_list):
    """
    Make possible use of spacy doc
    with already tokenized text
    """
    tokens = [t[0] for t in token_list]
    spaces = []
    for i, token in enumerate(tokens):
        if i == len(tokens)-1:
            spaces.append(False)
        elif ((tokens[i+1] in '!%),.:;?}')
              or (tokens[i+1].startswith("'") and len(tokens[i+1]) > 1)):
            spaces.append(False)
        else:
            spaces.append(True)
    assert len(spaces) == len(tokens)
    doc = spacy.tokens.doc.Doc(
        nlp.vocab, words=tokens, spaces=spaces)
    for name, proc in nlp.pipeline:
        doc = proc(doc)        
    return doc

In [8]:
def add_firsts(sent_list):
    """
    Add labels for first words in test data
    """
    new_sents = []
    for sent in sent_list:
        new_tokens = []
        for i, t in enumerate(sent):
            if i == 0:
                is_first = False
            elif sent[i-1][1]:
                is_first = True
            else:
                is_first = False
            triple = [t[0], t[1], is_first]
            new_tokens.append(triple)
        new_sents.append(new_tokens)
    return new_sents

In [9]:
#test_set = add_firsts(test_set)

In [7]:
PATH = '/mnt/hdd/Data/NLP/'
#ted = pd.read_csv(PATH+'transcripts.csv')

blogs = []
with open(PATH+'en_US/en_US.blogs.txt', 'r') as f:
    for line in f.readlines():
        if len(line) < 300:
            continue
        else:
            blogs.append(line.strip('\n '))

In [9]:
#ted['transcript'] = ted['transcript'].str.replace('\(.*?\)', ' ').str.strip()

План побудови фіч:

- розбиваємо текст на речення
- в кожному реченні виділяємо фічі для речення: довжина всіх символів, кількість слів, який токен закінчує речення
- переходимо на рівень слів і виділяємо фічі: саме слово, його номер у реченні, його довжина і форма, ПОС-таг, депенденсі-таг, чи є першим у реченні, чи є останнім, контекст - по два слова праворуч і ліворуч, всі їхні теги та інші фічі, ліва та права біграми;
- знаючи, який токен закінчує речення, повертаємось на рівень речень, беремо пари речень, видаляємо закінчуючий токен у першого з них, видаляємо фічу "кінець речення" в останнього, і розбиваємо все на біграми, кожна з яких міститиме фічі окремих слів (?), фічу "чи належать до одного чанку" (одної синтаксичної фрази), фічу "рівень найближчого батька", фічу "чи є один залежним від іншого"

In [12]:
def glue_sents(sents):
    """
    sents is a list of 2 to 3 sentences, most often 2
    returns list of tokens together with their first
    and last positions
    """
    glued_tokens = []
    for j, sent in enumerate(sents):
        doc = nlp(sent, disable=['parser', 'ner'])
        if len([t for t in doc if not t.is_punct]) == 0:
            continue
        for token in doc:
            if token.i == 0 and j != 0:
                # randomly uncapitalize some previously capitalized words
                if (random.random() < 0.5
                    and token.pos_ != 'PROPN'
                    and token.text != 'I'):
                    glued_tokens.append([token.text.lower(), False])
                else:
                    glued_tokens.append([token.text, False])
            elif (token.i == [t.i for t in doc 
                              if not t.is_punct][-1]
                  and j != (len(sents)-1)):
                glued_tokens.append([token.text, True])
            elif (j != (len(sents)-1) 
                  and token.is_punct
                  and token.i == (len(doc)-1)):
                continue
            else:
                glued_tokens.append([token.text, False])
    return glued_tokens

def glue_text(text):
    """
    Glue sentences in text in chunks of sizes 1, 2 and 3
    """
    sentences = sent_tokenize(text)
    text_len = len(sentences)
    current_ind = 0
    glued_tokens = []
    while current_ind < text_len:
        to_glue_ind = current_ind + random.choices([3, 4, 5], 
                        cum_weights=[35, 70, 100], k=1)[0]
        if to_glue_ind >= text_len:
            glued_tokens.append(glue_sents(sentences[current_ind:]))
        else:
            glued_tokens.append(glue_sents(sentences[current_ind:to_glue_ind]))
        current_ind = to_glue_ind
    return glued_tokens

blog_data = []
for text in tqdm(blogs):
    blog_data.extend(glue_text(text))
    
with open('blog_data.json', 'w') as wf:
    json.dump(blog_data, wf)

token_data = []
for text in tqdm(ted['transcript']):
    token_data.extend(glue_text(text))
    
with open('token_data.json', 'w') as wf:
    json.dump(token_data, wf)

In [8]:
ted_set = json.load(open(PATH+'ted_data.json'))
#blog_set = json.load(open(PATH+'blog_data.json'))

In [9]:
len(ted_set)

70273

In [10]:
random.seed = 477
ted_set = random.sample(ted_set, 10000)
#blog_set = random.sample(blog_set, 20000)

In [19]:
def get_features(token_data):
    """
    Make a list of features and two lists of labels
    from our data
    """
    data, labels = [], []
    wrong_count = 0
    for token_list in tqdm(token_data):
        doc = make_spaceable(token_list)
        sent_len = len(doc)
        lca_matrix = doc.get_lca_matrix()
        if not sent_len == len(token_list):
            continue
        for token in doc:
            i = token.i
            features = {
                'word': token.text,
                'word_lower': token.lower_,
                'word_lemma': token.lemma_,
                'word_pos': token.pos_,
                'word_dep': token.dep_,
                'word_capitalized': True if token.is_title else False,
                'word_ispunct': token.is_punct,
                'word_n_lefts': token.n_lefts,
                'word_n_rights': token.n_rights,
                'word_idx': token.idx,
                'length': len(token.text)
            }
            if i > 0:
                features.update({
                    'word-1': doc[i-1].lower_,
                    'w-1pos': doc[i-1].pos_,
                    'w-1dep': doc[i-1].dep_,
                    'word-1_capitalized': True if doc[i-1].is_title else False,
                    'word-1_ispunct': True if doc[-1].is_punct else False
                })
            if i > 1:
                features.update({
                    'word-2': doc[i-2].lower_,
                    'w-2dep': doc[i-2].dep_,
                    'left_bigram': doc[i-2].lower_ + '_' + doc[i-1].lower_
                })
            if i < sent_len - 1:
                lca = lca_matrix[i, i+1]
                features.update({
                    'word+1': doc[i+1].lower_,
                    'w+1pos': doc[i+1].pos_,
                    'w+1dep': doc[i+1].dep_,
                    'w+1_lca': lca if lca > -1 else 100,
                    'word+1_capitalized': True if doc[i+1].is_title else False,
                    'word+1_ispunct': True if doc[i+1].is_punct else False,
                    'word+1_n_rights': token.n_rights
                })
            if i < sent_len - 2:
                lca = lca_matrix[i, i+2]
                features.update({
                    'word+2': doc[i+2].lower_,
                    'w+2dep': doc[i+2].dep_,
                    'w+2_lca': lca if lca > -1 else 100,
                    'right_bigram': (doc[i+1].lower_ + '_' + doc[i+2].lower_)
                })
            last_lab = token_list[i][1]
            labels.append(last_lab)
            data.append(features)
    return data, labels

In [20]:
data, last_labels = get_features(ted_set)

100%|██████████| 10000/10000 [06:30<00:00, 25.59it/s]


In [13]:
del ted_set

In [21]:
test_set_features, test_lasts = get_features(test_set)

100%|██████████| 200/200 [00:01<00:00, 102.00it/s]


In [22]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data, last_labels, test_size=0.2, random_state=477)

In [23]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
train_features = vec.fit_transform(train_data)
test_features = vec.transform(test_data)
test_test_features = vec.transform(test_set_features)

In [24]:
clf = LogisticRegression(penalty='l1')
clf.fit(X=train_features, y=train_labels)
pred1 = clf.predict(test_features)
pred2 = clf.predict(test_test_features)

In [25]:
print(classification_report(test_labels, pred1, digits=3))

             precision    recall  f1-score   support

      False      0.989     0.996     0.993    159072
       True      0.875     0.708     0.783      5935

avg / total      0.985     0.986     0.985    165007



In [26]:
print(classification_report(test_lasts, pred2, digits=3))

             precision    recall  f1-score   support

      False      0.991     0.992     0.991      4542
       True      0.747     0.723     0.734       155

avg / total      0.982     0.983     0.983      4697



In [45]:
from sklearn.linear_model import SGDClassifier
clf2 = SGDClassifier(penalty='l2', loss='log', max_iter=1000, tol=1e-3)
clf2.fit(X=train_features, y=train_labels)
pred3 = clf2.predict(test_features)
print(classification_report(test_labels, pred3))

             precision    recall  f1-score   support

      False       0.98      0.99      0.99     79497
       True       0.68      0.47      0.56      2960

avg / total       0.97      0.97      0.97     82457



In [ ]:
pred4 = clf2.predict(test_test_features)
print(classification_report(test_lasts, pred4))

Тепер спробувати CRF

In [10]:
sent_full_data = [get_features([sent]) for sent in ted_set[:5000]]

In [11]:
sent_data = [data for (data, labels) in sent_full_data]
sent_labels = [labels for (data, labels) in sent_full_data]

In [13]:
def to_str(list_of_bools): return [str(b) for b in list_of_bools]
flatten = lambda l: [item for sublist in l for item in sublist]
sent_labels = [to_str(l) for l in sent_labels]
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(sent_data, sent_labels, test_size=0.2)
test_labels = flatten(test_labels)

In [14]:
from sklearn_crfsuite import CRF
crf = CRF(
    algorithm='lbfgs',
    c1=0.3,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(train_data, train_labels)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.3, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [15]:
pred3 = crf.predict(test_data)

In [16]:
pred3 = flatten(pred3)
print(classification_report(test_labels, pred3))

             precision    recall  f1-score   support

      False       0.99      1.00      0.99     77277
       True       0.84      0.71      0.77      2931

avg / total       0.98      0.98      0.98     80208



In [17]:
test_full_features = [get_features([sent]) for sent in test_set]
test_test_data = [data for (data, labels) in test_full_features]
test_test_labels = [labels for (data, labels) in test_full_features]
test_test_labels = [to_str(l) for l in test_test_labels]
pred4 = crf.predict(test_test_data)
pred4 = flatten(pred4)
test_test_labels = flatten(test_test_labels)
assert len(pred4) == len(test_test_labels)
print(classification_report(test_test_labels, pred4))

             precision    recall  f1-score   support

      False       0.99      0.99      0.99      4542
       True       0.71      0.69      0.70       155

avg / total       0.98      0.98      0.98      4697



Тут спроби зробити збалансовані дані:

In [35]:
data_zipped = np.array(list(zip(data, last_labels)))
data_t = data_zipped[np.array(last_labels)]
data_f = data_zipped[np.invert(last_labels)]
data_f_sample = random.sample(list(data_f), k=len(data_t))
data_sampled = random.sample(list(data_t) + list(data_f), k=2*len(data_t))
data_balanced = [x for (x,y) in data_sampled]
labels_balanced = [y for (x,y) in data_sampled]

In [36]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data_balanced, labels_balanced, test_size=0.3)
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
train_features = vec.fit_transform(train_data)
test_features = vec.transform(test_data)
test_test_features = vec.transform(test_set_features)

In [37]:
clf = LogisticRegression()
clf.fit(X=train_features, y=train_labels)
pred1 = clf.predict(test_features)
pred2 = clf.predict(test_test_features)

In [38]:
print(classification_report(test_labels, pred1))

             precision    recall  f1-score   support

      False       0.99      1.00      0.99      4653
       True       0.93      0.39      0.55        99

avg / total       0.99      0.99      0.98      4752



In [39]:
print(classification_report(test_lasts, pred2))

             precision    recall  f1-score   support

      False       0.98      1.00      0.99      4542
       True       0.84      0.40      0.54       155

avg / total       0.98      0.98      0.97      4697



Тут апсемплінг

In [18]:
from sklearn.utils import resample
train_true = np.array(train_data)[train_labels]
train_false = np.array(train_data)[np.invert(train_labels)]

In [37]:
train_false_labels = np.array(train_labels)[np.invert(train_labels)]

In [20]:
train_true = resample(train_true, n_samples=600000)

In [30]:
train_true_labels = np.full((600000,), True)

In [38]:
train_data = resample(list(zip(train_true, train_true_labels)) + list(zip(train_false, train_false_labels)))

In [41]:
train_labels = [x[1] for x in train_data]
train_data = [x[0] for x in train_data]

In [42]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
train_features = vec.fit_transform(train_data)
test_features = vec.transform(test_data)

In [43]:
# test_test is our 200 test sentences
test_test_features = vec.transform(test_set_features)

In [44]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
clf = LogisticRegression(penalty='l1', random_state=477)
clf.fit(X=train_features, y=train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=477, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [45]:
pred1 = clf.predict(test_features)
pred2 = clf.predict(test_test_features)

In [46]:
print(classification_report(test_labels, pred1))

             precision    recall  f1-score   support

      False       0.99      0.98      0.99    159254
       True       0.63      0.84      0.72      5830

avg / total       0.98      0.98      0.98    165084



In [47]:
print(classification_report(test_lasts, pred2))

             precision    recall  f1-score   support

      False       0.99      0.97      0.98      4542
       True       0.48      0.83      0.61       155

avg / total       0.98      0.97      0.97      4697

